In [68]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [69]:
data1 = pd.read_excel('U-GENE_data_RU_PL_fixed (1).xlsx')
data1 = data1[0:-1].drop(columns={'Unnamed: 0'})

In [70]:
data1 = data1.drop(columns=['ID', 'Impared fasting glucose (prediabetes)', 'Bilirubin bound, mkmol/L', 
                           'Uric acid, mmol/L', 'Glucose, mmol/L', 'α - lipoproteins, mg/dL (HDL)', 
                           ' β - lipoproteins, mmol/L (LDL)', 'Total cholesterol, mmol/L', 
                            'Triglycerides, mmol/L', 'Max life weight, kg.1', 'Birth weight, kg.1', 
                            'Weight one year ago, kg', 'Desired Weight, kg'])

data1 = data1.replace('<25', 25)
data1 = data1.replace('bd', 31)
data1 = data1.replace('ХИЛЕЗ', 4.12)
data1 = data1.replace('>350', 352)
data1 = data1.replace('`1', 1)
data1 = data1[~data1['Ethnicity'].isin([1, 2, 3, 4])].reset_index(drop=True)
data1['Ethnicity'] = data1['Ethnicity'].replace(5,1)

In [71]:
data1['BMI'] = data1['BMI'].astype(float)

In [72]:
data1['BMI'] = data1['BMI'].apply(lambda x: 1 if x > 29 else 0)

In [73]:
def change_target_dom(x):
    if x == 3:
        return 1
    elif x == 2:
        return 1
    else:
        return 0 

In [74]:
targets = ['Condition', 'Vascular diseases ','Diabetes',
       'Heart attack, chest pain, or other heart condition', 'Thyroid disease',
       'Dyslipidemia', 'Increased triglycerides', 'High blood pressure',
       'Increased fasting glucose', 'Metabolic Syndrome', 'BMI']

genetics = ['UCP1 rs45539933 (+1068GA)', 'UCP1 rs1800592  (-3826AG)',
       'UCP1 rs3811791 (-1766AG)', 'UCP1 rs10011540 (-112AC)',
       'UCP1 rs2270565 (Met229Leu)', 'UCP2 rs659366 (-866GA)', 'UCP2 rs660339 (Ala55Val)',
       'UCP3 rs1800849', 'FTO rs9939609', 'Ethnicity']

physical = ['Sex', 'Age', '\nHeight, сm', 'Weight, kg']

second_physical = ['Waist circumference, cm', 'WHR (waist to hip ratio)',
       'Hip circumference, cm', 'Neck circumference, cm', 'Musle mass, %', 
                  'Bone mass, kg', 'Visceral fat, kg']

blood =['Systolic BP', 'Diastolic BP ',   'Triglycerides, [mg/dL]', 'Total cholesterol, [mg/dL]',
       'α - lipoproteins, mmol/L (HDL)', 'VLDL, mmol/L',
       'β - lipoproteins, mg/dL (LDL)', 'Total pritein, g/L ', 'Albumin, g/L',
       'Fibrinogen, g/L', 'Uric acid, mg/dL', 'Creatinine, mg/dL',
       'Total bilirubin, mkmol/L', 'Glucose, mg/dL']

potential_removers = ['Max life weight, kg', 'Fat mass index, kg/m^2', 'BF%', 
                     'Neck circumference, cm']


additional_factors = ['The level of hydration, % ', 'Smoking history ',
       'Currently smoking ', 'Alcohol abuse ', 'History of sleep apnea', 
                      'History of joint pain/weakness',
       'History of muscle pain/weakness?', 
                     'The level of basal metabolism, kkal', 
                     'Physical rating', '20 years, weight, kg', 'Birth weight, kg', 'Ethnicity']

In [75]:
for el in genetics:
    data1[el] = data1[el].apply(lambda x: change_target_dom(x))

In [76]:
targets = ['Condition', 'Vascular diseases ','Diabetes',
       'Heart attack, chest pain, or other heart condition', 'Thyroid disease',
       'Dyslipidemia', 'Increased triglycerides', 'High blood pressure',
       'Increased fasting glucose', 'Metabolic Syndrome', 'BMI']

genetics = ['UCP1 rs45539933 (+1068GA)', 'UCP1 rs1800592  (-3826AG)',
       'UCP1 rs3811791 (-1766AG)', 'UCP1 rs10011540 (-112AC)',
       'UCP1 rs2270565 (Met229Leu)', 'UCP2 rs659366 (-866GA)', 'UCP2 rs660339 (Ala55Val)',
       'UCP3 rs1800849', 'FTO rs9939609', 'Ethnicity']

physical = ['Sex', 'Age', '\nHeight, сm', 'Weight, kg']

second_physical = ['Waist circumference, cm']

blood =['Systolic BP', 'Diastolic BP ',   'Triglycerides, [mg/dL]', 'Total cholesterol, [mg/dL]',
       'α - lipoproteins, mmol/L (HDL)', 
       'β - lipoproteins, mg/dL (LDL)','Uric acid, mg/dL',  'Glucose, mg/dL']

potential_removers = ['Max life weight, kg', 'Fat mass index, kg/m^2', 'BF%', 
                     'Neck circumference, cm', 'Albumin, g/L']


additional_factors =  ['Smoking history ',  'Alcohol abuse ', 'History of sleep apnea',
                       'History of joint pain/weakness',
                       'History of muscle pain/weakness?']

In [77]:
#data1 = data1.drop(columns=potential_removers+physical+ genetics+targets)

In [78]:
inf_full = []
inf_half = []

for el in data1.columns:
    if len(data1[~data1[el].isna()].values) > 900:
        inf_full.append(el)
    
    if len(data1[~data1[el].isna()].values) > 500:
        inf_half.append(el)

### Logistic Regression

In [79]:
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score, f1_score, average_precision_score

from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot  as plt
import numpy as np


from sklearn.metrics import classification_report

In [80]:
def get_prediction_log_reg(X, Y):

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
        X, Y, test_size=0.25, random_state = 45,  shuffle=True
    )
    print(Xtrain.shape)
    print(Xtest.shape)

    grid = {'C':np.logspace(-3,3,7), 
        'penalty':['none', 'l2']}

    model = LogisticRegression()

    logreg_cv = GridSearchCV(model, grid, cv=5, scoring='f1_macro')
    logreg_cv.fit(Xtrain, Ytrain)

    #print('Best parameters: ', logreg_cv.best_params_)
    #print('Best score: ', logreg_cv.best_score_)

    preds = logreg_cv.predict(Xtest)
    
    pred_metrics = {'accuracy':round(accuracy_score(Ytest, preds),3), 
                    'recall':round(recall_score(Ytest, preds), 3), 
                    'precision':round(precision_score(Ytest, preds), 3), 
                    'roc_auc_score':round(roc_auc_score(Ytest, preds), 3), 
                   'specificity':round(recall_score(np.logical_not(Ytest) , np.logical_not(preds)),3)}
    return pred_metrics

### Decision Trees

In [81]:
def get_prediction_dec_tree(X, Y):

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
        X, Y, test_size=0.25, random_state = 45, shuffle=True
    )
    print(Xtrain.shape)
    print(Xtest.shape)

    
    
    dec_tree = DecisionTreeClassifier(random_state=0)
    param_grid = {"max_depth": [40, 60, 80],
                  "min_samples_leaf": [1, 2],
                  "max_leaf_nodes": [180, 200, 220, 240],
                  }

    grid_cv_dtm = GridSearchCV(dec_tree, param_grid, cv=7, scoring='f1_macro')

    grid_cv_dtm.fit(Xtrain, Ytrain)
    #print('Best params: ', grid_cv_dtm.best_params_)
    #print('Best score: ', grid_cv_dtm.best_score_)

    preds_dec_grid = grid_cv_dtm.predict(Xtest)

    
    pred_metrics = {'accuracy':round(accuracy_score(Ytest, preds_dec_grid),3), 
                    'recall':round(recall_score(Ytest, preds_dec_grid), 3), 
                    'precision':round(precision_score(Ytest, preds_dec_grid), 3), 
                    'roc_auc_score':round(roc_auc_score(Ytest, preds_dec_grid), 3), 
                    'specificity':round(recall_score(np.logical_not(Ytest) , np.logical_not(preds_dec_grid)),3)}
    return pred_metrics

### Random Forest

In [82]:
def get_prediction_rf(X, Y):

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
        X, Y, test_size=0.25, random_state = 45,  shuffle=True
    )
    print(Xtrain.shape)
    print(Xtest.shape)

    

    param_grid = {
        'bootstrap': [True],
        'max_depth': [100, 110],
        'max_features': [2, 3],
        'min_samples_leaf': [4, 5],
        'min_samples_split': [8, 10],
        'n_estimators': [10, 20]
    }

    rf = RandomForestClassifier()
    grid_cv_rf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                              cv = 7, scoring='f1_macro')
    grid_cv_rf.fit(Xtrain, Ytrain)
    #print('Best params: ', grid_cv_rf.best_params_)
    #print('Best score: ', grid_cv_rf.best_score_)

    preds_rf_grid = grid_cv_rf.predict(Xtest)

    
    pred_metrics = {'accuracy':round(accuracy_score(Ytest, preds_rf_grid),3), 
                    'recall':round(recall_score(Ytest, preds_rf_grid), 3), 
                    'precision':round(precision_score(Ytest, preds_rf_grid), 3), 
                    'roc_auc_score':round(roc_auc_score(Ytest, preds_rf_grid), 3), 
                   'specificity':round(recall_score(np.logical_not(Ytest) , np.logical_not(preds_rf_grid)),3)}
    
    return pred_metrics

### SVM

In [83]:
def get_prediction_svm(X, Y):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(
        X, Y, test_size=0.25, random_state = 45, shuffle=True
    )
    print(Xtrain.shape)
    print(Xtest.shape)

    
    grid = {'kernel':['rbf', 'poly']}

    model = svm.SVC()
    # define the ovr strategy

    svm_cv = GridSearchCV(model, grid, cv=2, scoring='f1_macro') 
    svm_cv.fit(Xtrain, Ytrain)

    #print('Best parameters: ', svm_cv.best_params_)
    #print('Best score: ', svm_cv.best_score_)

    preds = svm_cv.predict(Xtest)
    
    pred_metrics = {'accuracy':round(accuracy_score(Ytest, preds),3), 
                    'recall':round(recall_score(Ytest, preds), 3), 
                    'precision':round(precision_score(Ytest, preds), 3), 
                    'roc_auc_score':round(roc_auc_score(Ytest, preds), 3), 
                   'specificity':round(recall_score(np.logical_not(Ytest) , np.logical_not(preds)),3)}

    
    return pred_metrics

In [84]:
import itertools

def get_combinations(input_list):
    result = []
    for r in range(1, len(input_list) + 1):
        combinations = list(itertools.combinations(input_list, r))
        result.extend(combinations)
    return result


items = [genetics, physical, second_physical, 
         blood, additional_factors]
combinations = get_combinations(items)

# Преобразуем кортежи в списки
combinations_as_lists = [list(comb) for comb in combinations]

print(len(combinations_as_lists))


31


In [85]:
final_combs = []
for el in combinations_as_lists:
    final_list = []
    for el2 in el:
        for k in el2:
            final_list.append(k)
    final_combs.append(final_list)

In [86]:
def main_prediction(df, features, target):
    
    final_features = []
    for el in features:
        for k in el.split('__'):
            final_features.append(k)
    
        
    
    filter_ = final_features + [target]
    df_main = df[filter_].dropna().reset_index(drop=True)
    
    if df_main.empty:
        print('df empty')
        return None
    
    X = df_main[final_features]
    Y = df_main[target]
    
    log_reg_metrics = get_prediction_log_reg(X, Y)
    dec_tree_metrics = get_prediction_dec_tree(X, Y)
    rf_metrics = get_prediction_rf(X, Y)
    svm_metrics = get_prediction_svm(X, Y)
    
    
    all_metrics = {}
    
    
    all_metrics['logreg'] = log_reg_metrics['roc_auc_score']
    all_metrics['dec_tree'] = dec_tree_metrics['roc_auc_score']
    all_metrics['rf'] = rf_metrics['roc_auc_score']
    all_metrics['svm'] = svm_metrics['roc_auc_score']
    return all_metrics
        

In [87]:
all_metrics_final = {}
for comb in final_combs:
    
    for tar in targets:
        
        print('Combination:  ' + '; '.join(comb))
        print('Target:  ' + tar)
        try:
            all_metrics_final[tar+ '~' +'+'.join(comb)] = main_prediction(data1, comb, tar)
        except:
            print('something went wrong')
            all_metrics_final[tar+ '~' +'+'.join(comb)] = None
        print('\n')

Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity
Target:  Condition


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity
Target:  Vascular diseases 


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity
Target:  Diabetes
something went wrong


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala5



Combination:  Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  High blood pressure


Combination:  Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Increased fasting glucose
something went wrong


Combination:  Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Metabolic Syndrome


Combination:  Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  BMI


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg
Target:  Condition


Combination:  UCP1 rs



Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  Diabetes
something went wrong


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  Heart attack, chest pain, or other heart condition


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826



Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Condition


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Vascular diseases 


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Diabetes
something went wrong


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Heart attack, chest pain, or other heart condition


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Thyroid disease


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Dyslipidemia


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Increased triglycerides


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  High blood pressure


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm
Target:  Increased fasting glucose


Combination:  Sex; Age; 
Height, 



Combination:  Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  Increased fasting glucose


Combination:  Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  Metabolic Syndrome


Combination:  Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  BMI


Combination:  Waist circumference, cm; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Condition


Combination:  Waist circumference, cm; Smoking history ; Alcohol abuse ; History of slee

...



Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Target:  Condition
something went wrong


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL
Targ



Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Increased triglycerides


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  High blood pressure


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1



Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  BMI


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); 

something went wrong


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Heart attack, chest pain, or other heart condition


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Thyroid disease


Combination:  Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total choleste



Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cholesterol, [mg/dL]; α - lipoproteins, mmol/L (HDL); β - lipoproteins, mg/dL (LDL); Uric acid, mg/dL; Glucose, mg/dL; Smoking history ; Alcohol abuse ; History of sleep apnea; History of joint pain/weakness; History of muscle pain/weakness?
Target:  Increased fasting glucose


Combination:  UCP1 rs45539933 (+1068GA); UCP1 rs1800592  (-3826AG); UCP1 rs3811791 (-1766AG); UCP1 rs10011540 (-112AC); UCP1 rs2270565 (Met229Leu); UCP2 rs659366 (-866GA); UCP2 rs660339 (Ala55Val); UCP3 rs1800849; FTO rs9939609; Ethnicity; Sex; Age; 
Height, сm; Weight, kg; Waist circumference, cm; Systolic BP; Diastolic BP ; Triglycerides, [mg/dL]; Total cho